In [4]:
from pprint import pprint
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [5]:
from lib_management.models import *

In [6]:
obj=Profile.objects.all()

In [7]:
obj

<QuerySet [<Profile: PIYUSH GUPTA>, <Profile: ayush>, <Profile: raja>]>

In [8]:
obj.filter(roles='user')

<QuerySet [<Profile: ayush>, <Profile: raja>]>

In [9]:
obj.filter(roles='user').first()

<Profile: ayush>

In [24]:
data = Author.objects.all()
ans = Author.objects.prefetch_related('book_details').all()
for author in ans:
    print(f"Author: {author.name}")
    for book_detail in author.book_details.all():
        print(f"  Book: {book_detail.name}")

Author: chandrbhan
Author: arjun singh
Author: narendra modi
  Book: jai shree ram
Author: jp nadda
Author: sonu nigam
  Book: jai shree ram
  Book: why this kolabary
Author: abdul kalam
  Book: wings of fire
Author: raja
  Book: veer tum bade chalo
Author: ayush
  Book: bade chalo
  Book: see-you-again
  Book: see-you-again
  Book: see-you-again
Author: charli
  Book: see-you-again
  Book: see-you-again
  Book: see-you-again
Author: chirag
  Book: see-you-again
  Book: see-you-again


In [28]:
ans = BookDetail.objects.prefetch_related('authors').all()
for book_detail in ans:
    print(f"Book: {book_detail.name}")
    print(book_detail.uid)
    for author in book_detail.authors.all():
        print(f"  Author: {author.name}")


Book: jai shree ram
f2ec7f88-8bd3-432a-bb79-270199c3633c
  Author: narendra modi
  Author: sonu nigam
Book: wings of fire
887965a4-1b63-425b-8d57-c169b9dead82
  Author: abdul kalam
Book: why this kolabary
a78cf42a-04ac-4948-8b11-ce75a1b36ab2
  Author: sonu nigam
Book: veer tum bade chalo
5cf679a5-f107-4e23-8967-a8f1fa7ad674
  Author: raja
Book: bade chalo
273acad8-b5af-44db-bbd0-0a6421becf05
  Author: ayush
Book: see-you-again
d6bfeb9d-982d-43b4-b4ed-048e3dce78b7
  Author: ayush
  Author: charli
  Author: chirag
Book: see-you-again
72e997c3-acc9-4956-b421-636c35865b08
  Author: ayush
  Author: charli
  Author: chirag
Book: see-you-again
0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c
  Author: ayush
  Author: charli


In [29]:
data=Author.objects.all()
for ans in data:
    print(ans.uid, ans.name)

2b272cdf-78f2-46c3-a684-250c36619896 chandrbhan
ce889b82-0ce6-4028-87da-9a0be8961ab7 arjun singh
8406a066-7752-4bd4-a0cc-32958032f91b narendra modi
e2ac632f-440d-4fc7-8faa-79897a2220b1 jp nadda
cb401bea-caa8-45d0-8049-b000fb4fa121 sonu nigam
38fadbfc-6bd3-4822-934f-bbed923878ae abdul kalam
ec71ca3b-560f-4e17-a452-02ca40363ff2 raja
dc9194fd-2c74-42da-b606-68a864bad52b ayush
7f220cdc-a3a9-40ee-a2ad-83599a045a75 charli
333b3b7e-e765-456b-a501-1ba36ee6dd83 chirag


In [13]:
Task: 1

Creat a Book-Detail-Handler class which will have these functions: 

create-book-detail(data: dictionary)
Data = {
   “name”="jai-ho"
   “authors”=["raja","ayush"]
}


SyntaxError: invalid character '“' (U+201C) (1384669714.py, line 7)

In [14]:
class Book_Detail_Handler:
    
    @staticmethod
    def create_book_details(data):
        try:
            # Validate data
            if not data.get('name') or not data.get('authors'):
                return {"message": "data is missing"}

            # Create the BookDetail
            book_details = BookDetail.objects.create(name=data['name'])

            # Create or get Author instances and add to the book
            for author_name in data['authors']:
                author, created = Author.objects.get_or_create(name=author_name)
                book_details.authors.add(author)

            return {"message": "Book details saved successfully",
                     "data": data}

        except Exception as e:
            return {"message": str(e)}
      
    
    @staticmethod
    def update_book_detail(book_id, data):
        try:
            if not data.get('name') or not data.get('authors') or not book_id:
                return {"message": "data is missing"}
            
            # Find the book record based on book_id
            book_update = BookDetail.objects.get(uid=book_id)
            book_update.name = data['name']
            book_update.save()  # Save the name update first
            
            # Clear existing authors
            book_update.authors.clear()
            
            # Add new authors
            for author_name in data['authors']:
                author, created = Author.objects.get_or_create(name=author_name)
                book_update.authors.add(author)
            
            return {"message": "Book details updated successfully",
                    "data" : data
                   }


        except Exception as e:
            return {"message": str(e)}
            
     
    
    @staticmethod
    def retrieve_book_detail(book_id):
        try:
            if not book_id:
                return {"message": "book_id is missing"}
            
            #retrive the book details based on book_id
            book_retrive=BookDetail.objects.get(uid=book_id)
            
            authors=[]
            for auth in book_retrive.authors.all():
                authors.append(auth.name)
                
                
            response={
                "name":book_retrive.name,
                "authors":authors,
            }
            
            return {"message": "Book details retrive Successfully",
                     "data": response
                   }
        except Exception as e:
            return {"message": str(e)}
                
            
            
    @staticmethod        
    def delete_book_detail(book_id):
        try:
            if not book_id:
                return {"message": "book_id is missing"}
            
            #delete book details based on uid
            book_delete=BookDetail.objects.get(uid=book_id)
            
            authors = [auth.name for auth in book_delete.authors.all()]
                
                
            response={
                "name":book_delete.name,
                "authors":authors,
            }
            
            book_delete.delete()
            
            return {"message": "Book details deleted Successfully",
                     "data": response
                   }
        
        
        except Exception as e:
            return {"message": str(e)}
        
        
        


In [15]:
data = {
    "name": "see-you-again",
    "authors": ["ayush","charli","chirag"]
}



response = Book_Detail_Handler.create_book_details(data)
print(response)

{'message': 'Book details saved successfully', 'data': {'name': 'see-you-again', 'authors': ['ayush', 'charli', 'chirag']}}


In [68]:
data = {
    "name": "see-you-again",
    "authors": ["ayush","charli","chirag"]
}
id = '319e20f7-e84c-493f-8b48-79936c6f7734'  
response = Book_Detail_Handler.update_book_detail(id, data)
print(response)

{'message': 'BookDetail matching query does not exist.'}


In [67]:
id = '319e20f7-e84c-493f-8b48-79936c6f7734'
response=Book_Detail_Handler.retrieve_book_detail(id)
print(response)

{'message': 'BookDetail matching query does not exist.'}


In [72]:
id = 'c44aa409-f457-44cb-9252-80c9448cb33d'
response=Book_Detail_Handler.delete_book_detail(id)
print(response)

{'message': 'Book details deleted Successfully', 'data': {'name': 'jai-ho', 'authors': ['raja', 'ayush']}}
